In [ ]:
# gui/main_window.py
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from models.database import Database
from gui.add_employee_dialog import AddEmployeeDialog
from gui.edit_employee_dialog import EditEmployeeDialog

class MainWindow(tk.Tk):
    def __init__(self, admin=True):
        super().__init__()
        self.title("Payroll System")
        self.geometry("800x500")
        self.db = Database()
        self.admin = admin
        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self)
        frame.pack(fill="both", expand=True, padx=10, pady=10)

        columns = ["ID", "Name", "Email", "Pay Type", "Status"]
        self.tree = ttk.Treeview(frame, columns=columns, show="headings")
        for col in columns:
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True)

        if self.admin:
            ctrl = ttk.Frame(frame)
            ctrl.pack(fill="x", pady=5)
            ttk.Button(ctrl, text="Add Employee", command=self.add_employee_dialog).pack(side="left", padx=5)
            ttk.Button(ctrl, text="Edit Selected", command=self.edit_selected).pack(side="left", padx=5)
            ttk.Button(ctrl, text="Delete Selected", command=self.delete_selected).pack(side="left", padx=5)
            ttk.Button(ctrl, text="Search", command=self.search_dialog).pack(side="left", padx=5)
            ttk.Button(ctrl, text="Calculate Payroll (Lock)", command=self.calculate_payroll).pack(side="right", padx=5)
            ttk.Button(ctrl, text="Generate Report", command=self.generate_report).pack(side="right", padx=5)

        self.populate_employee_list()

    def populate_employee_list(self):
        for r in self.tree.get_children():
            self.tree.delete(r)
        for e in self.db.list_employees():
            self.tree.insert("", "end", values=(e.employee_id, f"{e.first_name} {e.last_name}", e.email, e.pay_type, e.status))

    def add_employee_dialog(self):
        AddEmployeeDialog(self, self.db)
        self.populate_employee_list()

    def edit_selected(self):
        sel = self.tree.focus()
        if sel:
            eid = self.tree.item(sel, "values")[0]
            emp = self.db.search_employee(eid)
            if emp:
                EditEmployeeDialog(self, self.db, emp)
                self.populate_employee_list()
        else:
            messagebox.showwarning("Select", "No employee selected.")

    def delete_selected(self):
        sel = self.tree.focus()
        if sel:
            eid = self.tree.item(sel, "values")[0]
            if messagebox.askyesno("Confirm", f"Delete employee {eid}?"):
                self.db.delete_employee(eid)
                self.populate_employee_list()

    def search_dialog(self):
        term = simpledialog.askstring("Search", "Enter Employee ID or Email:")
        if term:
            emp = self.db.search_employee(term)
            if emp:
                messagebox.showinfo("Found", f"{emp.first_name} {emp.last_name}, Email: {emp.email}")
            else:
                messagebox.showinfo("Not Found", "Employee not found")

    def calculate_payroll(self):
        messagebox.showinfo("Payroll", "Payroll calculated (demo).")

    def generate_report(self):
        messagebox.showinfo("Report", "Report generated (demo).")